In [1]:
# %pip install tqdm

In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [5]:
max_audio_length_seconds = 10
sample_rate = 44100


In [6]:
def extract_features_cnn(file_name):
    try:
        audio, _ = librosa.load(file_name, sr=sample_rate)
        
        # Calculate the number of frames needed for a 10-second audio clip
        num_frames = int(np.ceil(max_audio_length_seconds * sample_rate / 512))  # Assuming n_fft=512
        
        # Adjust the number of mel filters
        n_mels = min(40, int(np.floor(sample_rate / 700)))  # Limit to 40 or lower
        
        # Extract MFCCs with adjusted parameters
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40, n_fft=512, hop_length=512, n_mels=n_mels)

        # Pad or truncate to ensure a fixed number of frames
        if mfccs.shape[1] > num_frames:
            mfccs = mfccs[:, :num_frames]
        else:
            pad_width = num_frames - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
        print(f"Error encountered while parsing the file {file_name}: {str(e)}")
        return None

    return mfccs

In [7]:
def create_dataset(path, column_name):
    features = []
    
    class_labels = {'Real': 0, 'Fake': 1}
    
    # Iterate over the range of the maximum number of files in any folder
    max_files = max(len(os.listdir(os.path.join(path, label))) for label in class_labels)
    
    for i in tqdm(range(max_files)):
        for label in class_labels:
            label_path = os.path.join(path, label)
            class_label = class_labels[label]
            
            files = os.listdir(label_path)
            
            # Check if the current index is within the range of files for this label
            if i < len(files):
                filename = files[i]
                file_path = os.path.join(label_path, filename)
                data = extract_features_cnn(file_path)
                features.append([data, class_label])
        
    featuresdf = pd.DataFrame(features, columns=['feature', 'class_label'])
    return featuresdf


In [8]:
# Audio files directory
audio_files_path = 'Dataset'
# Create the dataset
dataset = create_dataset(audio_files_path, column_name='class_label')

# Display the first few rows of the dataset
print(dataset.head())


  0%|          | 0/2892 [00:00<?, ?it/s]

100%|██████████| 2892/2892 [02:29<00:00, 19.40it/s]


                                             feature  class_label
0  [[-423.01138, -408.51053, -395.1726, -388.9639...            0
1  [[-242.95776, -300.41302, -294.46365, -304.463...            1
2  [[-199.06863, -222.71597, -198.40753, -202.447...            0
3  [[-303.8373, -399.50528, -402.58997, -403.6783...            1
4  [[-270.0309, -304.7558, -320.5852, -328.39526,...            0


In [21]:
dataset.shape

(5380, 2)

In [3]:
new = pd.read_pickle("dataset.csv")


In [4]:
new['feature'][0]

array([[-4.2301138e+02, -4.0851053e+02, -3.9517261e+02, ...,
        -4.2344995e+02, -4.2254727e+02, -4.2308609e+02],
       [ 6.0483599e-01,  1.2950470e+01,  2.6960434e+01, ...,
         0.0000000e+00,  1.2692488e+00,  5.1102626e-01],
       [ 5.5946714e-01,  1.1654037e+01,  2.4354542e+01, ...,
         0.0000000e+00,  1.2472196e+00,  5.0037491e-01],
       ...,
       [ 9.8930404e-02, -1.4684772e-01, -7.5094604e-01, ...,
         0.0000000e+00, -2.0234507e-01, -1.7811027e-01],
       [ 6.4966381e-02,  1.6016536e+00,  2.2719297e+00, ...,
         0.0000000e+00, -1.3621956e-01, -1.2012987e-01],
       [ 3.2153279e-02, -2.2537074e+00, -2.9430456e+00, ...,
         0.0000000e+00, -6.8506360e-02, -6.0484186e-02]], dtype=float32)

In [5]:
print(new.head())

                                             feature  class_label
0  [[-423.01138, -408.51053, -395.1726, -388.9639...            0
1  [[-242.95776, -300.41302, -294.46365, -304.463...            1
2  [[-199.06863, -222.71597, -198.40753, -202.447...            0
3  [[-303.8373, -399.50528, -402.58997, -403.6783...            1
4  [[-270.0309, -304.7558, -320.5852, -328.39526,...            0


In [20]:
# Convert the 'feature' column to strings
dataset['feature'] = dataset['feature'].astype(str)

# Save the dataset as an HDF file
hdf_filename = 'dataset.h5'
dataset.to_hdf(hdf_filename, key='dataset', mode='w', format='table', complevel=9, complib='zlib')

# Display the first few rows of the dataset
print(dataset.head())


                                             feature  class_label
0  [[-4.2301138e+02 -4.0851053e+02 -3.9517261e+02...            0
1  [[-2.4295776e+02 -3.0041302e+02 -2.9446365e+02...            1
2  [[-1.99068634e+02 -2.22715973e+02 -1.98407532e...            0
3  [[-3.0383731e+02 -3.9950528e+02 -4.0258997e+02...            1
4  [[-2.7003091e+02 -3.0475580e+02 -3.2058521e+02...            0


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into 90% training and 10% test
X_train, X_temp, y_train, y_temp = train_test_split(new['feature'], new['class_label'], test_size=0.2, random_state=42)

# Further split the test data into 50% validation and 50% testing
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [8]:
test_dataframe = pd.concat([X_test, y_test], axis =1 )

In [10]:
csv_filename = 'img/test.csv'
test_dataframe .to_pickle(csv_filename)

In [11]:
train_dataframe = pd.concat([X_train, y_train], axis=1)


In [12]:
train_dataframe['feature'][0].shape

(40, 862)

In [13]:
csv_filename = 'img/train.csv'
train_dataframe .to_pickle(csv_filename)

In [15]:
val_dataframe = pd.concat([X_validation, y_validation], axis=1)


In [37]:
test_dataframe

feature  class_label
705   [[-341.36246, -386.00787, -379.26157, -379.135...            1
4788  [[-272.42377, -308.24585, -290.03055, -282.273...            0
1212  [[-289.31628, -358.8922, -354.65863, -327.7315...            0
1379  [[-485.84164, -485.70337, -485.5972, -485.6013...            1
5370  [[-443.5221, -436.77887, -340.86923, -314.8520...            1
...                                                 ...          ...
745   [[-240.9313, -237.42084, -247.05226, -290.8520...            1
3688  [[-306.28363, -305.10236, -305.97318, -305.666...            0
1434  [[-198.88222, -214.30157, -168.74835, -139.744...            0
4822  [[-433.67026, -433.67026, -433.67026, -433.670...            0
195   [[-432.18182, -418.9554, -429.10858, -437.0991...            1

[1076 rows x 2 columns]

In [16]:
test_dataframe['feature'].iloc[0].shape

(40, 862)

In [17]:
csv_filename = 'img/validate.csv'
val_dataframe .to_pickle(csv_filename)